# Permutation Tests

## Basic idea: 

If, under the null hypothesis, the probability distribution of the data is invariant under the action of some group $\mathcal{G}$, then once we observe the actual data, we know other possible data sets that are equally likely.

In particular, every element of the _orbit_ of the observed data under $\mathcal{G}$ was just as probable as the
observed data.

Test _conditionally_ on the orbit in which the observed data lie.

## Example: Effect of treatment in a randomized controlled experiment

11 pairs of rats, each pair from the same litter.  

Randomly&mdash;by coin tosses&mdash;put one of each pair into
"enriched" environment; other sib gets "normal" environment.

After 65 days, measure cortical mass (mg).

<table>
<tr>
<td> enriched </td><td> 689 </td><td> 656 </td><td> 668 </td><td> 660 </td><td> 679 </td><td> 663 </td><td> 664 </td><td> 647 </td><td> 694 </td><td> 633 </td><td>653 </td>
</tr>
<tr>
<td> impoverished </td><td> 657</td><td> 623</td><td> 652</td><td> 654</td><td> 658</td><td> 646</td><td> 600</td><td> 640</td><td> 605</td><td> 635</td><td> 642 </td>
</tr>
<tr>
<td> difference </td><td> 32</td><td>  33</td><td>  16</td><td>   6</td><td>  21</td><td>  17</td><td>  64</td><td>   7</td><td>  89</td><td>  -2</td><td>  11
</tr>
</table>

**How should we analyze the data?**

Cartoon of Rosenzweig, M.R., E.L. Bennet, and M.C. Diamond, 1972. Brain changes in response to experience, _Scientific American_, _226_, 22&ndash;29 report an experiment in which 11 triples of male rats, each triple from the same litter, were
assigned at random to three different environments, "enriched" (E), standard, and "impoverished."
See also Bennett et al., 1969. 

### Informal Hypotheses

Null hypothesis: treatment has "no effect."

Alternative hypothesis: treatment increases cortical mass.

Suggests 1-sided test for an increase.

### Test contenders

+ 2-sample Student $t$-test: 
$$ 
              \frac{\mbox{mean(treatment) - mean(control)}}
                   {\mbox{pooled estimate of SD of difference of means}}
$$
+ 1-sample Student $t$-test on the differences: 
$$
	      \frac{\mbox{mean(differences)}}{\mbox{SD(differences)}/\sqrt{11}}
$$  
Better, since littermates are presumably more homogeneous.
              
+ Permutation test using $t$-statistic of differences:
same statistic, different way to calculate $P$-value.
Even better?

### Strong null hypothesis

**Treatment has no effect whatsoever---as if cortical mass were 
assigned to each rat before the randomization.**

Then equally likely that the rat with the heavier cortex will be assigned
to treatment or to control, independently across littermate pairs.

Gives $2^{11} = 2,048$ equally likely possibilities:

<table>
<tr>
<td>difference </td><td> $\pm$32 </td><td> $\pm$33 </td><td> $\pm$16 </td><td> $\pm$6 </td><td> $\pm$21 </td><td> $\pm$17 </td><td> $\pm$64 </td><td> $\pm$7 </td><td> $\pm$89 </td><td> $\pm$2 </td><td> $\pm$11 </td>
</tr>
</table>

For example, just as likely to observe original differences as

<table>
<tr>
	<td>difference </td><td>-32 </td><td> -33 </td><td> -16 </td><td> -6 </td><td> -21 </td><td> -17 </td><td>
    -64</td><td> -7 </td><td> -89 </td><td> -2 </td><td> -11</td>
</tr>
</table>



### Weak null hypothesis

On average across pairs, treatment makes no difference.

### Alternatives

1. Individual's response depends only on that individual's assignment
    + Special cases: shift, scale, etc.

1. Interactions/Interference: my response could depend on whether you are assigned to treatment or control.

### Assumptions of the tests

1. 2-sample $t$-test: 
    + masses are iid sample from normal distribution, same unknown variance, same unknown mean.
	+ Tests weak null hypothesis (plus normality, independence, non-interference, etc.).
1. 1-sample $t$-test on the differences: 
	+ mass differences are iid sample from normal distribution, unknown variance, zero mean.
	+ Tests weak null hypothesis (plus normality, independence, non-interference, etc.)
1. Permutation test: 
	+ Randomization fair, independent across pairs.
	+ Tests strong null hypothesis.

Assumptions of the permutation test are true by design: That's how treatment
was assigned.

### What's the group?

The data are elements of $\Re^{11}$. The group operations reflect coordinates
about the axes&mdash;they flip the signs of elements of the vectors.

In [1]:
# boilerplate
%matplotlib inline
from __future__ import division
import math
import numpy as np
import scipy as sp
from scipy import stats  # distributions
from scipy import special # special functions
from scipy import random # random variables, distributions, etc.
from scipy.optimize import brentq
from scipy.stats import (binom, hypergeom)
import matplotlib.pyplot as plt
from ipywidgets import widgets

treat = np.array([689, 656, 668, 660, 679, 663, 664, 647, 694, 633, 653])
control = np.array([657, 623, 652, 654, 658, 646, 600, 640, 605, 635, 642])
diffr = treat - control

sp.stats.ttest_1samp(diffr, 0.0)  # two-sided one-sample t-test

Ttest_1sampResult(statistic=3.2437214037805222, pvalue=0.0088136932072599358)

### Student $t$-test calculations

Mean of differences:  26.73mg  
Sample SD of differences:  27.33mg  
$t$-statistic: $26.73/(27.33/\sqrt{11}) = 3.244$. 

$P$-value for 2-sided $t$-test:  0.0088

+ Why do cortical weights have normal distribution?
+ Why is variance of the difference between treatment and control the same for different litters?
+ Treatment and control are _dependent_ because assigning a rat to treatment excludes it from the control group, and vice versa.
+ $P$-value depends on assuming differences are iid sample from a normal distribution.
+ If we reject the null, is that because there is a treatment effect, or because the other assumptions are wrong?

### Permutation $t$-test calculations

Could enumerate all $2^{11} = 2,048$ equally likely possibilities.
Calculate $t$-statistic for each.

$P$-value is 
$$
	P = \frac{\mbox{number of possibilities with $t \ge 3.244$}}{\mbox{2,048}}
$$
(Using the mean as the test statistic instead of $t$-statistic
would yield $P = 2/2048 =  0.00098$.)

If there were many more pairs, it would be impractical to enumerate, but 
we can simulate:

Assign a random sign to each difference. \\
Compute $t$-statistic \\
Repeat $N$ times
$$
	P \approx \frac{\mbox{number of simulations with $t \ge 3.244$}}{N}
$$
Can invert Binomial tests to make an exact confidence interval for $P$ from the simulation results.

In [2]:
np.random.RandomState(seed=20151013)  # set seed for reproducibility

iter = 100000  # N
def simPermuTP(z, iter):
# P.B. Stark, statistics.berkeley.edu/~stark 
# simulated P-value for 2-sided 1-sample t-test under the 
# randomization model.
    tsf = lambda(x): np.abs(np.mean(x)/(np.std(x, ddof=1)/math.sqrt(len(x))))
    ts = tsf(z) 
    return np.mean([ (tsf(z*(2*np.random.random_integers(0, 1, size=len(z))-1)) >= ts) for i in range(iter)])
    
estP = simPermuTP(diffr, iter)
print estP, estP*iter

0.00237 237.0


In [3]:
# confidence interval for the p-value
# This function is in the _permute_ library; see http://statlab.github.io/permute/

def binom_conf_interval(n, x, cl=0.975, alternative="two-sided", p=None,
                        **kwargs):
    """
    Compute a confidence interval for a binomial p, the probability of success in each trial.

    Parameters
    ----------
    n : int
        The number of Bernoulli trials.
    x : int
        The number of successes.
    cl : float in (0, 1)
        The desired confidence level.
    alternative : {"two-sided", "lower", "upper"}
        Indicates the alternative hypothesis.
    p : float in (0, 1)
        Starting point in search for confidence bounds for probability of success in each trial.
    kwargs : dict
        Key word arguments

    Returns
    -------
    tuple
        lower and upper confidence level with coverage (approximately)
        1-alpha.

    Notes
    -----
    xtol : float
        Tolerance
    rtol : float
        Tolerance
    maxiter : int
        Maximum number of iterations.
    """
    assert alternative in ("two-sided", "lower", "upper")

    if p is None:
        p = x / n
    ci_low = 0.0
    ci_upp = 1.0

    if alternative == 'two-sided':
        cl = 1 - (1-cl)/2

    if alternative != "upper" and x > 0:
        f = lambda q: cl - binom.cdf(x - 1, n, q)
        ci_low = brentq(f, 0.0, p, *kwargs)
    if alternative != "lower" and x < n:
        f = lambda q: binom.cdf(x, n, q) - (1 - cl)
        ci_upp = brentq(f, 1.0, p, *kwargs)

    return ci_low, ci_upp

binom_conf_interval(iter, math.ceil(estP*iter), cl=0.95, alternative="two-sided", p=estP)

(0.0020875068460931615, 0.0027019248405985314)

In [8]:
binom_conf_interval(10**5, 907, cl=0.99, alternative="upper")

(0.0, 0.009792108717906225)

### Other tests: sign test, Wilcoxon signed-rank test

#### Sign test:
Count pairs where treated rat has heavier cortex, i.e., where
difference is positive.

Under strong null, distribution of the number of positive differences
is Binomial(11, 1/2).  Like number of heads in 11 independent tosses
of a fair coin.  (Assumes no ties w/i pairs.)

$P$-value is chance of 10 or more heads in 11 tosses of a fair coin: 0.0059.

Only uses signs of differences, not information that only the smallest absolute 
difference was negative.

#### Wilcoxon signed-rank test
uses information about the ordering of the
differences: rank the absolute values of the differences, then give them
the observed signs and sum them.  Null distribution: assign signs at random
and sum.

### Still more tests, for other alternatives

All the tests we've seen here are sensitive to _shifts_--the alternative
hypothesis is that treatment increases response (cortical mass).

There are also nonparametric tests that are sensitive to other
treatment effects, e.g., treatment increases the variability of the 
response.

And there are tests for whether treatment has any effect at all on
the distribution of the responses.

You can design a test statistic to be sensitive to any change that
interests you, then use the permutation distribution to get a $P$-value
(and simulation to approximate that $P$-value).

### Silliness

Treat ordinal data (e.g., Likert scale) as if measured on a linear scale; 
use Student $t$-test.

Maybe not so silly for large samples$\ldots$

$t$-test asymptotically distribution-free.

How big is big?

## Back to Rosenzweig et al.

Actually had 3 treatments:  enriched, standard, deprived.

Randomized 3 rats per litter into the 3 treatments, independently across
$n$ litters.

**How should we analyze these data?**



### Test contenders

$n$ litters, $s$ treatments (sibs per litter).
+ ANOVA--the $F$-test:
$$ 
F = \frac{\mbox{BSS}/(s-1)}{\mbox{WSS}/(n-s)}
$$
+ Permutation $F$-test: use permutation distribution instead of $F$ distribution to get $P$-value.
+ Friedman test: Rank within litters. Mean rank for treatment $i$ is $\bar{R}_i$.
$$ 
Q = \frac{12n}{s(s+1)} \sum_{i=1}^s \left ( \bar{R}_i - \frac{s+1}{2} \right )^2.
$$
$P$-value from permutation distribution. 

### Strong null hypothesis

**Treatment has no effect whatsoever&mdash;as if cortical mass were 
assigned to each rat before the randomization.**

Then it's equally likely that each littermate is assigned to each treatment, 
independently across litters.

There are $3! = 6$ assignments of each triple to treatments. Thus, there $6^n$ equally likely assignments across all litters.

For 11 litters, that's 362,797,056 possibilities.

(The group under which the distribution of the data is invariant is a subgroup of the permutation group: if you think of the data as an element of $\Re^{33}$ (33 rats), the group consists of all permutations of the 33 elements that shuffle only adjacent sets of 3 components: components 1&ndash;3, components 4&ndash;6, &hellip; components 31&ndash;33.)


### Weak null hypothesis

**The average cortical weight for all three treatment groups are equal.
On average across triples, treatment makes no difference.**

### Assumptions of the tests

+ $F$-test: masses are iid sample from normal distribution, same unknown variance, same unknown mean for all litters and treatments.  
Tests weak null hypothesis.
+ Permutation $F$-test: Randomization was as advertised: fair, independent across triples.  
Tests strong null hypothesis.
+ Friedman test: Ditto.

Assumptions of the permutation test and Friedman test are true by design: 
that's how treatment was assigned.

Friedman test statistic has  $\chi^2$ distribution asymptotically.  Ties are a complication.

### $F$-test assumptions&mdash;reasonable?

+ Why do cortical weights have normal distribution for each litter and for each treatment?
+ Why is the variance of cortical weights the same for different litters?
+ Why is the variance of cortical weights the same for different treatments?

Is $F$ a good statistic for this alternative?

$F$-test (and Friedman statistic) sensitive to differences among the 
mean responses for each treatment group, no matter what pattern the differences 
have.

But the treatments and the responses can be ordered: we hypothesize that
more stimulation produces greater cortical mass.

deprived $\Longrightarrow$  normal  $\Longrightarrow$  enriched   
low mass  $\Longrightarrow$  medium mass  $\Longrightarrow$  high mass

_Can we use that to make a more sensitive test?_

### A test against an ordered alternative

Within each litter triple, count pairs of responses
that are "in order."  Sum across litters.

E.g., if one triple had cortical masses

<table>
<tr>
<td>deprived </td><td> 640 </td>
</tr>
<tr>
<td>normal </td><td> 660 </td>
</tr>
<tr>
<td>enriched </td><td>  650 </td>
</table>

that would contribute 2 to the sum: $660 \ge 640$, $650 \ge 640$, but $640 < 650$.

Each litter triple contributes between 0 and 3 to the overall sum.


Null distribution for the test based on the permutation distribution: 6
equally likely assignments per litter, independent assignments across litters.

### A different test against an ordered alternative

Within each litter triple, add differences
that are "in order."  Sum across litters.

E.g., if one triple had cortical masses

<table>
<tr>
<td>deprived </td><td> 640 </td>
</tr>
<tr>
<td>normal </td><td> 660 </td>
</tr>
<tr>
<td>enriched </td><td>  650 </td>
</table>

that would contribute 30 to the sum: $660 - 640 = 20$ and $650 - 640 = 10$, but $640 < 650$,
so that pair contributes zero.

Each litter triple contributes between 0 and $2\times{\mbox{ range }}$ to the sum.

Null distribution for the test based on the permutation distribution: 6
equally likely assignments per litter, independent across litters.

## The 2-sample problem 

Suppose we have a group of $N$ individuals who are randomized into two groups, a _treatment_ group of size $N_t$ and a _control_ group of size $N_c = N - N_t$.
Label the individuals from $1$ to $N$.
Let ${\mathcal T}$ denote the labels of individuals assigned to treatment and ${\mathcal C}$ denote 
the labels of those assigned to control.

For each of the $N$ individuals, we measure a quantitative (real-valued) response.
Each individual $i$ has two _potential responses_: the response $c_i $individual would have if assigned to 
the control group, and the response $t_i$ the individual would have if assigned to the treatment group.

We assume that individual $i$'s response depends _only_ on that individual's assigment, and not on anyone else's assignment.
This is the assumption of _non-interference_. 
In some cases, this assumption is reasonable; in others, it is not.

For instance, imagine testing a vaccine for a communicable disease.
If you and I have contact, whether you get the disease might depend on whether I am vaccinated&mdash;and _vice versa_&mdash;since if the vaccine protects me from illness, I won't infect you.
Similarly, suppose we are testing the effectiveness of an advertisement for a product.
If you and I are connected and you buy the product, I might be more likely to buy it, even if I don't
see the advertisement.

Conversely, suppose that "treatment" is exposure to a carcinogen, and the response whether the
subject contracts cancer. 
On the assumption that cancer is not communicable, my exposure and your disease
status have no connection.

The _strong null hypothesis_ is that individual by individual, treatment makes no difference whatsoever: $c_i = t_i$ for all $i$.

If so, any differences between statistics computed for the treatment and control groups are entirely due to the luck of the draw: which individuals happened to be assigned to treatment and which to control.

We can find the _null distribution_ of any statistic computed from the responses of the two groups: if the strong null hypothesis is true, we know what individual $i$'s response would have been whether assigned to treatment or to control&mdash;namely, the same.

For instance, suppose we suspect that treatment tends to increase response: in general, $t_i \ge c_i$.
Then we might expect $\bar{c} = \frac{1}{N_c} \sum_{i \in {\mathcal C}} c_i$ to be less than
$\bar{t} = \frac{1}{N_t} \sum_{i \in {\mathcal T}} t_i$.
How large a difference between $\bar{c}$ and $\bar{t}$ would be evidence that treatment increases the response,
beyond what might happen by chance through the luck of the draw?

This amounts to asking whether the observed difference in means between the two groups is a high percentile
of the distribution of that difference in means, calculated on the assumption that the null hypothesis is true.

Because of how subjects are assigned to treatment or to control, all allocations of $N_t$ subjects to
treatment are equally likely.

One way to partition the $N$ subjects randomly into a group of size $N_c$ and a group of size $N_t$ is
to permute the $N$ subjects at random, then take the first $N_c$ in the permuted list to be the control
group, and the remaining $N_t$ to be the treatment group.

## Gender Bias in Teaching Evaluations
MacNell, Driscoll, and Hunt (2014. [What's in a Name: Exposing Gender Bias in Student Ratings of Teaching](http://link.springer.com/article/10.1007%2Fs10755-014-9313-4), _Innovative Higher Education_) conducted a controlled, randomized experiment on
the effect of students' perception of instructors' gender on teaching evaluations
in an online course.
Students in the class did not know the instructors' true genders.

MacNell et al. randomized 47 students in an online course into four groups: two taught by a female
instructor and two by a male instructor.
One of the groups taught by each instructor was led to believe the instructor was male;
the other was led to believe the instructor was female.
Comparable instructor biographies were given to all students.
Instructors treated the groups identically, including returning assignments at the same time.

When students thought the instructor was female, the responding students rated the instructor lower, on average,
in every regard.

<table>
<tr><th>Characteristic</th> <th>F - M</th></tr>
<tr><td>Caring</td><td> -0.52</td></tr>
<tr><td> Consistent</td><td> -0.47</td></tr>
<tr><td> Enthusiastic</td><td> -0.57</td></tr>
<tr><td> Fair </td><td>-0.76</td></tr>
<tr><td> Feedback</td><td> -0.47</td></tr>
<tr><td> Helpful</td><td> -0.46</td></tr>
<tr><td> Knowledgeable</td><td> -0.35</td></tr>
<tr><td> Praise</td><td> -0.67</td></tr>
<tr><td> Professional</td><td> -0.61</td></tr>
<tr><td> Prompt</td><td> -0.80</td></tr>
<tr><td> Respectful</td><td> -0.61</td></tr>
<tr><td> Responsive</td><td> -0.22</td></tr>
</table>

Those results are for a 5-point scale, so a difference of 0.8 (Promptness) is 16% of the entire scale.

MacNell et al. graciously shared their data.
The evaluation data are coded as follows:

    Group
          3 (12 students) - TA identified as male, true TA gender female 
	      4 (11 students) - TA identified as male, true TA gender male
	      5 (11 students, 8 responders) - TA identified as female, true TA gender female
	      6 (13 students, 12 responders) - TA identified as female, true TA gender male
    tagender - 1 if TA is actually male, 0 if actually female 
    taidgender - 1 if TA is identified as male, 0 if identified as female 
    gender - 1 if student is male, 0 if student is female

There are grades for 47 students but evaluations for only 43 (4 did not respond: three in group 5 and one in group 6). 
The grades are not linked to the evaluations, per the IRB protocol.

Our null hypothesis will include the assumption that whether a student responds does not depend on the identified gender of the TA: the 4 nonresponders would have remained nonresponders had they been in that instructor's other section, and no additional students would become nonresponders if they had been assigned to the same instructor's other section.

### Interlude: partitioning sets into more than two subsets

Recall that there are $n \choose k$ ways of picking a subset of size $k$ from $n$ items;
hence there are $n \choose k$ ways of dividing a set into a subset of size $k$ and one of size $n-k$
(once you select those that belong to the subset of size $k$, the rest must be in the complementary
subset of size $n-k$.

In this problem, we are partitioning 47 things into 4 subsets, two of size 11, one of size 12, and
one of size 13.
How many ways are there of doing that?


Recall the [Fundamental Rule of Counting](http://www.stat.berkeley.edu/~stark/SticiGui/SticiGui/Text/counting.htm#fundamental_rule): 
If a set of choices, $T_1, T_2, \ldots,  T_k$, could result, respectively, 
in $n_1, n_2, \ldots, n_k$ possible outcomes, the entire set of $k$ choices has
$\prod_{i=1}^k n_k$ possible outcomes.

We can think of the allocation of students to the four groups as choosing 11 of the 47
students for the first group, then 11 of the remaining 36 for the second, 
then 12 of the remaining 25 for the third.
The fourth group must contain the remaining 13.

The number of ways of doing that is

$$  
   {47 \choose 11}{36 \choose 11}{25 \choose 12} =
   \frac{47!}{11! 36!} \frac{36!}{11! 25!} \frac{25!}{12! 13!} = \frac{47!}{11! 11! 12! 13!}.
$$

Does the number depend on the order in which we made the choices?
Suppose we made the choices in a different order: first 12 students for one group, then
11 for the second, then 13 for the third (the fourth gets the remaining 11 students).
The number of ways of doing that is
$$  
   {47 \choose 12}{35 \choose 11}{24 \choose 13} =
   \frac{47!}{12! 35!} \frac{35!}{11! 24!} \frac{24!}{13! 11!} = \frac{47!}{12! 11! 13! 11!} = .
$$
The number does not depend on the order in which we make the choices.

By the same reasoning, the number of ways of dividing a set of $n$ objects into
$m$ subsets of sizes $k_1, \ldots k_m$ is given by the _multinomial coefficient_
$$
   {n \choose k_1, k_2, \ldots, k_m} =
   {n \choose k_1}{n-k_1 \choose k_2} {n-k_1-k_2 \choose k_3} \cdots {n - \sum_{i=1}^{m-1} k_i \choose k_{m-1}}
$$

$$ = \frac{n! (n-k_1)! (n-k_1 - k_2)! \cdots 
   (n - k_1 - \cdots - k_{m-1}!}{k_1! (n-k_1)! k_2! (n-k_1-k_2)! \cdots k_m!}
   = \frac{n!}{\prod_{i=1}^m k_i!}.
$$

We will check how surprising it would be for the means to be so much lower when the TA is identified as female, if in fact there is "no real difference" in how they were rated, and the apparent difference is just due to the luck of the draw: which students happened to end up in which section.

In the actual randomization, all $47 \choose 11, 11, 12, 13$ allocations
were equally likely.
But there might be real differences between the two instructors.
Hence, we'd like to use each of them as his or her own "control."

Each student's potential responses are represented by a ticket with 4 numbers:

+ the rating that the student would assign to instructor A if instructor A is identified as male
+ the rating that the student would assign to instructor A if instructor A is identified as female
+ the rating that the student would assign to instructor B if instructor B is identified as male
+ the rating that the student would assign to instructor B if instructor B is identified as female

The null hypothesis is that the first two numbers are equal and the second two numbers are equal,
but the first two numbers might be different from the second two numbers.
This corresponds to the hypohtesis that  
students assigned to a given TA would rate him or her the same, whether that TA seemed to be male or female.
For all students assigned instructor A, we know both of the first two numbers if the hull hypothesis
is true; but we know neither of the second two numbers.
Similarly, if the null hypothesis is true, we know both of the second two numbers for all students
assigned to instructor B, but we know neither of the first two numbers for those students.

Because of how the randomization was performed, all allocations 
of students to sections that keep the number of students in each section the same are equally likely, so
in particular all allocations that keep the same students assigned to each actual instructor
the same are equally likely.
This is a subgroup of the full permutation group.

Hence, all ${23 \choose 12}$ ways of splitting the 23 students assigned to the female instructor into two groups, one with 11 students and one with 12, are equally likely. Similarly, all
${24 \choose 11}$ ways of splitting the 24 students assigned to the male instructor into two groups, one with 13 students and one with 11, are equally likely.
We can thus imagine shuffling the female TA's students between her sections, and the male TA's students
between his sections, and examine the distribution of the difference between the mean score for the sections where the
TA was identified as male is larger than the mean score for the sections where the TA was identified as
female.
(The 4 nonresponders are also allocated at random between sections of the same 
instructor; they are not included in the averages.)

If the difference is rarely as large as the observed mean difference, the observed mean difference gives
evidence that being identified as female really does lower the scores.

In [11]:
# Number of allocations to 11, 11, 12, 13
print sp.special.binom(47,11)*sp.special.binom(36,11)*sp.special.binom(25,12)  # big number!

5.44175311066e+25


In [5]:
# the data are in a .csv file called "Macnell-RatingsData.csv" in the directory Data
import pandas as pd

ratings = pd.read_csv("./Data/Macnell-RatingsData.csv")
categories = ratings.columns.values.tolist()[1:15]
print ratings.head()
print ratings.describe()

   group  professional  respect  caring  enthusiastic  communicate  helpful  \
0      3             5        5       4             4            4        3   
1      3             4        4       4             4            5        5   
2      3             5        5       5             5            5        5   
3      3             5        5       5             5            5        3   
4      3             5        5       5             5            5        5   

   feedback  prompt  consistent  fair  responsive  praised  knowledgeable  \
0         4       4           4     4           4        4              3   
1         5       5           3     4           5        5              5   
2         5       5           5     5           5        5              5   
3         5       5           5     5           3        5              5   
4         5       3           4     5           5        5              5   

   clear  overall  gender   age  tagender  taidgender  
0     

In [6]:
from permute.core import corr, two_sample, permute_within_groups
from permute.utils import get_prng
rs = np.random.RandomState(seed=1)  # set the seed of the PRNG, for reproducibility

In [7]:
def stratified_two_sample(x, y, group_x, group_y, stat='mean', alternative="greater", reps=10**4, 
                          keep_dist=False, seed=None):
    """
    One-sided or two-sided, two-sample permutation test for equality of
    two means, with p-value estimated by simulated random sampling with
    reps replications.

    Tests the hypothesis that x and y are a random partition of x,y
    against the alternative that x comes from a population with mean

    (a) greater than that of the population from which y comes,
        if side = 'greater'
    (b) less than that of the population from which y comes,
        if side = 'less'
    (c) different from that of the population from which y comes,
        if side = 'two-sided'

    Permutations are carried out within the given groups.  Under the null hypothesis,
    observations within each group are exchangeable.

    If ``keep_dist``, return the distribution of values of the test statistic;
    otherwise, return only the number of permutations for which the value of
    the test statistic and p-value.

    Parameters
    ----------
    x : array-like
        Sample 1
    y : array-like
        Sample 2
    group_x : array-like
        Group assignments for sample 1
    group_y : array-like
        Group assignments for sample 2
    stat : {'mean', 't'}
        The test statistic.

        (a) If stat == 'mean', the test statistic is (mean(x) - mean(y))
            (equivalently, sum(x), since those are monotonically related), omitting
            NaNs, which therefore can be used to code non-responders
        (b) If stat == 't', the test statistic is the two-sample t-statistic--
            but the p-value is still estimated by the randomization,
            approximating the permutation distribution.
            The t-statistic is computed using scipy.stats.ttest_ind
        (c) If stat is a function (a callable object), the test statistic is
            that function.  The function should take a permutation of the pooled
            data and compute the test function from it. For instance, if the
            test statistic is the Kolmogorov-Smirnov distance between the
            empirical distributions of the two samples, max_t |F_x(t) - F_y(t)|,
            the test statistic could be written:

            f = lambda u: np.max( \
                [abs(sum(u[:len(x)]<=v)/len(x)-sum(u[len(x):]<=v)/len(y)) for v in u]\
                )        
    alternative : {'greater', 'less', 'two-sided'}
        The alternative hypothesis to test
    reps : int
        Number of permutations
    keep_dist : bool
        flag for whether to store and return the array of values
        of the irr test statistic
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator.

    Returns
    -------
    float
        the estimated p-value
    float
        the test statistic
    list
        The distribution of test statistics.
        These values are only returned if `keep_dist` == True        
    """
    
    prng = get_prng(seed)
    
    z = np.concatenate([x, y])   # pooled responses
    groups = np.concatenate([group_x, group_y])   # pooled group assignments
    
    # If stat is callable, use it as the test function. Otherwise, look in the dictionary
    stats = {
        'mean': lambda u: np.nanmean(u[:len(x)]) - np.nanmean(u[len(x):]),
        't': lambda u: ttest_ind(
            u[:len(y)][~np.isnan(u[:len(y)])], 
            u[len(y):][~np.isnan(u[len(y):])], 
            equal_var=True)[0]
    }
    if callable(stat):
        tst_fun = stat
    else:
        tst_fun = stats[stat]

    theStat = {
        'greater': tst_fun,
        'less': lambda u: -tst_fun(u),
        'two-sided': lambda u: math.fabs(tst_fun(u))
    }
    tst = theStat[alternative](z)
    observed_tst = tst_fun(z)
    
    if keep_dist:
        dist = np.empty(reps)
        for i in range(reps):
            dist[i] = theStat[alternative](permute_within_groups(z, groups, seed=prng))
        hits = np.sum(dist >= tst)
        return hits/reps, tst, dist
    else:
        hits = np.sum([(theStat[alternative](permute_within_groups(z, groups, seed=prng)) >= tst)
                       for i in range(reps)])
        return hits/reps, tst

In [8]:
(p, t) = stratified_two_sample(ratings['overall'][ratings.taidgender==1], ratings['overall'][ratings.taidgender==0], 
                               ratings['tagender'][ratings.taidgender == 1], 
                               ratings['tagender'][ratings.taidgender == 0],
                               alternative = "two-sided", seed = rs)
print 'Overall rating:'
print 'Difference in means:', t
print 'P-value (two-sided):', np.round(p, 5), "\n"

print ('\n\n{0:24} {1:8} {2:8}'.format('Category', 'Diff means', 'P-value'))
for col in categories:
    (p, t) = stratified_two_sample(ratings[col][ratings.taidgender==1], ratings[col][ratings.taidgender==0], 
                               ratings['tagender'][ratings.taidgender == 1],
                               ratings['tagender'][ratings.taidgender == 0],
                               alternative = "two-sided", seed = rs)
    print ('{0:20} {1:8.2f} {2:8.2f}'.format(col, t, p))

Overall rating:
Difference in means: 0.473913043478
P-value (two-sided): 0.1187 



Category                 Diff means P-value 
professional             0.61     0.06
respect                  0.61     0.06
caring                   0.52     0.09
enthusiastic             0.57     0.06
communicate              0.57     0.07
helpful                  0.46     0.17
feedback                 0.47     0.16
prompt                   0.80     0.01
consistent               0.46     0.21
fair                     0.76     0.01
responsive               0.22     0.48
praised                  0.67     0.02
knowledgeable            0.35     0.29
clear                    0.41     0.30


## General test based on group invariance

References:

+ Romano, J.P., 1988. A bootstrap revival of some nonparametric distance tests, _J. Amer. Stat. Assoc., 83_, 698&ndash;708
+ Romano, J.P., 1989. Bootstrap and randomization tests of some nonparametric hypotheses, _Ann. Stat., 17_, 141&ndash;159

Data $X \sim P$ takes values in $\mathcal X$.

$\mathcal G$ is a finite group of transformations from $\mathcal X$ to $\mathcal X$.
$\# \mathcal G = G$.

Want to test null hypothesis $H_0: P \in \Omega_0$.

Suppose $H_0$ implies that $P$ is invariant under $\mathcal G$:
$$
   \forall g \in \mathcal{G}, \;\; X \sim gX.
$$

The _orbit of $x$ (under $\mathcal G$)_ is $\{ gx : g \in \mathcal G \}$.
(Does the orbit of $x$ always contain $G$ points?)

### Test statistic $T$

Let $T: \mathcal{X} \rightarrow \Re$ be a test statistic.

We want to test $H_0$ at significance level $\alpha$.

For each fixed $x$, let $T^{(k)}(x)$ be the $k$th smallest 
element of the multiset
$$
    [ T(gx): g \in \mathcal G ].
$$

These are the $G$ (not necessarily distinct) values $T$ takes on the orbit of $x$.

### Finding the rejection region

Let
$$
     r \equiv G - \lfloor \alpha G \rfloor.
$$

Define
$$
     G^+(x) \equiv \# \{ g \in \mathcal{G}: T(gx) > T^{(r)}(x) \}
$$
and
$$
     G^r(x) \equiv \# \{ g \in \mathcal{G}: T(gx) = T^{(r)}(x) \}
$$

Let
$$
      a(x) \equiv \frac{\alpha G - G^+(x)}{G^r(x)}.
$$

Define
$$
   \phi(x) \equiv 
        \left \{ 
            \begin{array}{ll}
                 1,      &  T(x) > T^{(r)}(x), \cr
                a(x),   &  T(x) = T^{(r)}(x), \cr
                 0,      &  T(x) < T^{(r)}(x)
            \end{array}
         \right .
$$

To test the hypothesis, generate $U \sim U[0, 1]$ independent of $X$.

Reject $H_0$ if $\phi(x) \ge U$.  (Randomized test.)


### Test has level $\alpha$ unconditionally

For each $x \in \mathcal{X}$,

$$
    \sum_{g \in \mathcal{G}} \phi(gx) = G^+(x) + a(x) G^r(x) = \alpha G.
$$

So if $X \sim gX \sim P$ for all $g \in \mathcal G$,
$$
\alpha  =  \mathbb{E}_P \frac{1}{G}\sum_{g \in \mathcal {G}} \phi(gX) 
$$
$$
 =  \frac{1}{G}\sum_{g \in \mathcal{G}} \mathbb{E}_P \phi(X) 
$$
$$
 =  \mathbb{E}_P \phi(X).
$$

The unconditional chance of a Type I error is exactly $\alpha$.


### Tests for the mean of a symmetric distribution

Data $X = (X_j )_{j=1}^N \in \mathcal{X} = \Re^n$.

$\{ X_j \}$ iid $P$; $\mathbb{E} X_j = \mu$.

Suppose $P$ is symmetric.  Examples?


### Reflection group

Let $\mathcal{G}_\mu$ be the group of reflections of coordinates about $\mu$.

Let $x \in \Re^n$.
Each $g \in \mathcal{G}_\mu$ is of the form

$$
g(x) = (\mu + (-1)^{\gamma_j}(x_j - \mu))_{j=1}^n
$$

for some $\gamma = (\gamma_j)_{j=1}^n \in \{0, 1\}^n$.

Is $\mathcal{G}_\mu$ really a group?

What's the identity element?  
What's the inverse of $g$?  
What $\gamma$ corresponds to $g_1g_2$?

What is $G$, the number of elements of $\mathcal{G}_\mu$?

What is the orbit of a point $x$ under $\mathcal{G}_\mu$?  
Are there always $2^n$ distinct elements of the orbit?

Test statistic:

$$
   T(X) = | \bar{X} - \mu_0 | = \left | \frac{1}{n} \sum_{j=1}^n X_j - \mu_0 \right |.
$$

If $\mathbb{E} X_j = \mu_0$, this is expected to be small---but how 
large a value would be surprising?


If the expected value of $X_j$ is $\mu$ and $P$ is symmetric (i.e., if $H_0$
is true), 
the $2^n$ potential data

$$
   \{ gX : g \in \mathcal{G}_\mu \}
$$

in the orbit of $X$ under $\mathcal{G}$ are equally likely.

Hence, the values in the multiset

$$
   [ T(gx) : g \in \mathcal{G}_\mu ]
$$

are equally likely, conditional on the event that $X$ is in the orbit of $x$.

### How to test $H_0$: $\mu = \mu_0$?

We observe $X = x$.

If fewer than $\alpha G$ values in $[T(gx) : g \in \mathcal{G}_{\mu_0}]$ are greater than 
or equal to
$T(x)$, reject.
If more than $\alpha G$ values are greater than $T(x)$, don't reject.
Otherwise, randomize.


### If $n$ is big &hellip;

How can we sample at random from the orbit?

Toss fair coin $n$ times in sequence, independently.
Take $\gamma_j = 1$ if the $j$th toss gives heads; $\gamma_j = 0$ if tails.

Amounts to sampling with replacement from the orbit of $x$.

### Other test statistics?

Could use $t$-statistic, but calibrate critical value using the permutation distribution.

Could use a measure of dispersion around the hypothesized mean (the
true mean minimizes expected RMS difference, assuming variance is finite).

What about counting the number of values that are above $\mu_0$?

Define 
$$
T(x) \equiv \sum_{j=1}^n 1_{x \ge \mu_0}.
$$

### Sign test for the median

We are assuming $P$ is symmetric, so the expected value and median are equal.

To avoid unhelpful complexity, suppose $P$ is continuous.

Then 

$$
\Pr \{ X_j \ge \mu \} = 1/2, 
$$

$$
   \{ 1_{X_j \ge \mu} \}_{j=1}^n \mbox{ are iid Bernoulli}(1/2),
$$

and 

$$
    T(X) \sim \mbox{Binomial}(n, 1/2).
$$

This leads to the sign test:  Reject the hypothesis that the median is $\mu_0$
if $T(X)$ is too large or too small.
Thresholds set to get level $\alpha$ test, using the fact that $T(X)\sim \mbox{Binomial}(n, 1/2)$.


### Is the sign test equivalent to the permutation test for the same test statistic?

Suppose we are using the test statistic $T(x) = \sum_{j=1}^n 1_{x \ge \mu_0}$.
Do the permutation test and the sign test reject for the same values of $x \in \mathcal{X}$?

Suppose no component of $x$ is equal to $\mu_0$.
According to the sign test, the chance that $T(x) = k$ is
${{n}\choose{k}} 2^{-n}$ if the null is true.

What's the chance that $T(x) = k$ according to the permutation test?
There are $G = 2^n$ points in the orbit of $x$ under $\mathcal G$.
If the null is true, all have equal probability $2^{-n}$.
Of these points, ${{n}\choose{k}}$ have $k$ components with positive deviations from $\mu_0$.
Hence, for the permutation test, the chance that $T(x) = k$ is also 
${{n} \choose {k}} 2^{-n}$:  The two tests are equivalent.


### Abstract setting using VC classes

The distribution $P \in \Omega$, where $\Omega$ is a known collection of
distributions on $\mathcal{X}$.
The null hypothesis is that $P \in \Omega_0 \subset \Omega$.  We assume that
$\Omega_0$ can be characterized as a set of distributions that are invariant under
a transformation on $\Omega$: let $\tau: \Omega \rightarrow \Omega_0$; we assume
that $\tau(P) = P$ for all $P \in \Omega_0$.

Let $\mathcal{V}$ be a collection of subsets of a set $\mathcal{X}$.
For a finite set $D \subset \mathcal{X}$, let $\Delta^\mathcal{V}(D)$ be the number of distinct sets
$\{ V \cap D: V \in \mathcal{V} \}$.
For positive integers $n$, let
$$
    m^\mathcal{V}(n) = \max_{D \subset \mathcal{X}: \#D = n } \Delta^\mathcal{V}(D).
$$

Let
$$
    c(\mathcal{V}) \equiv \inf \{ n : m^\mathcal{V}(n) < 2^n \}.
$$

If $c(\mathcal{V}) < \infty$, $\mathcal{V}$ is a Vapnik-Cervonenkis (V-C) class.
That is, $\mathcal{V}$ is a V-C class
if the maximum number of distinct intersections of sets in $\mathcal{V}$
with sets containing $n$ points grows sub-exponentially with $n$.
Intersections, finite unions, and Cartesian products of V-C classes are V-C classes.
In $\Re^n$, the set of all ellipsoids, the set of all half-spaces, the set of all
lower-left quadrants, and the set of all convex sets with at most $p$ extreme
points are all V-C classes.

An alternative, equivalent definition of a V-C class is based on the following definition:

_Suppose $\mathcal{V}$ is a collection of subsets of a set $\mathcal{X}$, and that $D$ is a finite subset of $\mathcal{X}$.
We say $D$ is _shattered_ by $\mathcal{V}$ if every subset $d \subset D$
can be written $d = V \cap D$ for some $V \in \mathcal{V}$._

Suppose $D$ has $n$ elements.
Because there are $2^n$ subsets of a set with $n$ elements, this is equivalent to saying
that there are $2^n$ different subsets of the form $D \cap V$ as $V$ ranges over $\mathcal{V}$.

A collection $\mathcal{V}$ is a V-C class if for some finite integer $n$, there exists a set
$D \subset \mathcal{X}$ with $n$ elements that is not shattered by $\mathcal{V}$.

__Example__. Half lines on $\Re$.
Consider a set $D = \{x_j\}_{j=1}^n$ of points on the real line.
Let $\mathcal{V} = \{ (-\infty, y] : y \in \Re \}$.
How many sets are there  of the form $V \cap D$, for $V \in \mathcal{V}$?
Just $n+1$.
Suppose the points are in increasing order, so that $x_1 < x_2 < \cdots < x_n$.
Then the possibilities for $V \cap D$ are $\{ \}$,
$\{ x_1 \}$, $\{ x_1, x_2 \}$, $\ldots$, $\{x_j \}_{j=1}^n$.
Thus $m^{\mathcal{V}}(n) = n+1$, and $c(\mathcal{V}) \equiv \inf \{ n : m^\mathcal{V}(n) < 2^n \} = 2$
(for $n=0$, we have $0+1 = 2^0$, and for $n=1$,
we have $1+1 = 2^1$, but for $n=2$, we have $2+1 < 2^2$).

__Example__. Closed intervals $\{[y, z]$: $y < z\}$ on $\Re$.
For finite sets $D$ as discussed above,
the possibilities for $V \cap D$ include all sets of adjacent values, such as
$\{x_1\}$, $\{x_2\}$, $\{x_3\}$, $\{x_1, x_2\}$, $\{x_2, x_3\}$, and $\{x_1, x_2, x_3\}$, but not,
for example, $\{ x_1, x_3 \}$.
Clearly, $m^{\mathcal{V}}(2) = 4$ but $m^{\mathcal{V}}(3) = 7$, so $c(\mathcal{V}) = 3$.
(The general rule is $m^{\mathcal{V}}(n) = 1 + n + {n}\choose{2}$. Why?)

Suppose that $\mathcal{V}$ and $\mathcal{W}$ are V-C classes on a common set $\mathcal{X}$.
Then $\mathcal{V} \cup \mathcal{W}$ is also a V-C class, as is $\mathcal{V} \cap \mathcal{W}$.

### Exercise

+ Show that intersections and finite unions of V-C classes are V-C classes.

+ Show by example that a countable union of V-C classes need not be a V-C class.

Let $\mathcal{V}$ be a VC
class of subsets of $\mathcal{S}$.
Define the pseudo-metric
$$
\delta: \Omega \times \Omega \rightarrow \Re^+
$$
$$
(P, Q)  \rightarrow  \sup_{V \in \mathcal{V}} | P(V) - Q(V) | .
$$

This is a generalization of the Kolmogorov-Smirnov distance for distributions on the
line.
In that case, the sets in $\mathcal{V}$ are the half-lines $\{(-\infty, y] : y \in \Re\}$
(which comprise a
V-C class).

Assume that $\mathcal{V}$ and $\tau$ have been selected such that
$\delta(P, \tau P) = 0$ iff
$P \in \Omega_0$.
Romano proposes using the test statistic
$$
    T_n = n^{1/2} \delta(\hat{P}_n, \tau \hat{P}_n ),
$$
where $\hat{P}_n$ is the empirical measure of $\{X_j \}_{j=1}^n$.
One rejects the hypothesis when  $\tau\hat{P}_n$ is far from $\hat{P}_n$; i.e.,
when $T_n$ is sufficiently large.

But how large?
One way to obtain a critical value for the test is with the bootstrap:
resample from $\tau(\hat{P}_n)$, tabulate the distribution of the distance
between the empirical distribution of the bootstrap samples and $\tau$ applied to them,
use the $1-\alpha$ quantile of that distribution as the critical value for an
approximate level $\alpha$ test.
(We have to resample from $\tau(\hat{P}_n)$ rather than $\hat{P}_n$ because the
significance level is computed under the assumption that the null hypothesis is _true_.
The null hypothesis is true for $\tau(\hat{P}_n)$ but not necessarily for $\hat{P}_n$.)

Suppose that there is a (known) group $\mathcal{G}_n$ of transformations of the sample space
$\mathcal{S}_n$
such that under the null hypothesis, $P$ is invariant under $\mathcal{G}_n$.
Then we can also construct a _randomization test_ of the hypothesis $H_0$.
For simplicity, suppose that $\mathcal{G}_n$ is finite, with $M_n$ elements
$\{ g_{nj} \}_{j=1}^{M_n}$.
Under the null hypothesis, conditional on $X = x$, the values $\{ g_{nj}x \}_{j=1}^{M_n}$
are equally likely.
(The _orbit_ of an point $x$ in a space $S$ acted on by a group $\mathcal{G}$ is the set of
all elements of $S$ that can be obtained by applying elements of $\mathcal{G}$ to $x$.
That is, it is the set $\{ g(x): g \in \mathcal{G}\}$.
For example, consider points in the plane and the group of rotations about the
origin.
Then the orbit of a point $x$ is the circle with radius $\|x\|$.)

Compute the  test statistic for each $ g_{nj}x$ in the orbit of $x$.
Reject the null hypothesis if the statistic for $x$ exceeds the $1-\alpha$ quantile of
the test statistic for the set of values obtained from the orbit;
do not reject if it is less; reject with a given probability if the statistic equals
the $1-\alpha$ quantile, in such a way as to get a level $\alpha$ test.
This is a randomization test.
Because the level of the randomization test is $\alpha$, conditional on the data,
integrating
over the distribution of the data shows that it is $\alpha$ unconditionally.

### Examples of hypotheses and functions $\tau$

Examples Romano gives include testing for independence of the components of each $X_j$,
testing for exchangeability of the components of each $X_j$, testing for spherical
symmetry of the distribution of $X_j$, testing for homogeneity among the $X_j$, and
testing for a change point.

In the example of testing for independence, the mapping $\tau$ takes the marginal
distributions of the joint distribution, then constructs a joint distribution that
is the product of the marginals.  For distributions with independent components,
this is the identity; otherwise, it maps a distribution into one with the same
marginals, but whose components are independent.
For testing for spherical symmetry, $\tau$ maps a distribution into one with the same  mass
at every distance from the origin, but that is uniform on spherical shells.
For testing for exchangability, Romano proposes looking at the largest difference between
$P$ and a permutation of the coordinates of $P$, over all permutations of the coordinates.
See his paper for more details.

Romano shows that these tests are consistent against all alternatives, and that
the critical values given by the bootstrap and by randomization are asymptotically
equal with probability one.  Because the randomization tests are exact level $\alpha$
tests, they might be preferred.  Romano also briefly discusses how to implement the tests
computationally.

Let's consider the implementation in more detail, for two hypotheses:
independence of the components of a $k$-variate distribution, and
rotational invariance of a bivariate distribution.

### Independence
We observe $\{X_j\}_{j=1}^n$ iid $P$, where each $X_j = (X_{ij})_{i=1}^k$ takes values in $\Re^k$.
Under the null hypothesis, $P$ is invariant under the mapping $\tau$ that takes the $k$
marginal distributions of $P$ and multiplies them together to give a probability on
$\Re^k$ with independent components.
Let $\hat{P}_n$ be the empirical measure; let the V-C class $\mathcal{V}$ be the set of
lower left quadrants $\{ Q(x) : x \in \Re^k \}$ where
$$
    Q(x) \equiv \{ y \in \Re^k : y_i \le x_i, \;\; i = 1, \ldots, k\}.
$$
Then
$$
    \hat{P}_n(Q(x)) = \frac{1}{n} \# \{ X_j : X_{ij} \le x_i, \;\; i = 1, \ldots, k \},
$$
and
$$
    \tau \hat{P}_n (Q(x)) = \prod_{i=1}^k \frac{1}{n} \# \{ X_j : X_{ij} \le x_i \}.
$$
The maximum difference in the probability of a lower left quadrant $Q(x)$ occurs when
$x$ is one of the points of support of $\tau \hat{P}_n$:
$$
\sup_{V \in \mathcal{V}} | \hat{P}_n (V) - \tau \hat{P}_n(V) |
= \sup_{x \in \Re^k} | \hat{P}_n (Q(x)) - \tau \hat{P}_n(Q(x)) |
$$
$$
= \max_{x \in \Re^k : x_i \in \{X_{ij}\}_{j=1}^n, \;\; i = 1, \ldots, k }
| \hat{P}_n (Q(x)) - \tau \hat{P}_n(Q(x)) |.
$$

To test the null hypothesis of independence, we would compute
$$
    T(X) = \max_{x \in \Re^k : x_i \in \{X_{ij}\}_{j=1}^n, \;\; i = 1, \ldots, k }
    | \hat{P}_n (Q(x)) - \tau \hat{P}_n(Q(x)) |
$$
from the data $X$, then repeatedly draw iid samples $X^*$ of size $n$ from $\tau \hat{P}_n$,
computing
$$
    T(X^*) = \max_{x \in \Re^k : x_i \in \{X_{ij}^*\}_{j=1}^n, \;\; i = 1, \ldots, k }
    | \hat{P}_n^* (Q(x)) - \tau \hat{P}_n^*(Q(x)) |
$$
for each.
We would reject the null hypothesis
that the components of $P$ are independent (at approximate significance level $\alpha$) if
$T(X)$ exceeds the $1-\alpha$ quantile of the simulated distribution of $T(X^*)$.

### Rotational invariance in $\Re^2$

We observe $\{X_j\}_{j=1}^n$ iid $P$, where each $X_j = (X_{1j}, X_{2j})$ takes values in $\Re^2$.

For $y \in \Re^2$, define $|y| \equiv \sqrt{y_1^2 + y_2^2}$ to be the distance from $y$ to
the origin.

Except at the origin, the mapping from Cartesian coordinates $(x_1, x_2)$ to polar coordinates
$(r, \theta)$ is one-to-one; identify
the origin with the polar coordinates $(0, 0)$.
Under the null hypothesis, $P$ is invariant under the mapping $\tau$ that produces
a distribution with the same marginal distribution of $|X|$ but that is uniform on
$\theta$ for each possible value of $|X|$.

As before, let $\hat{P}_n$ be the empirical measure; let the V-C class $\mathcal{V}$ be the set of
lower left quadrants $\{ Q(x) : x \in \Re^2 \}$ where
$$
    Q(x) \equiv \{ y \in \Re^k : y_i \le x_i, \;\; i = 1, 2\}.
$$

Then
$$
    \hat{P}_n(Q(x)) = \frac{1}{n} \# \{ X_j : X_{ij} \le x_i, \;\; i = 1, 2 \}.
$$

To proceed, we need to find the probability of lower left quadrants $Q(x)$ for the
distribution $\tau \hat{P}_n$.
Consider the contribution from each $X_j$ separately.
Let $R_j = |X_j| = \sqrt{X_{1j}^2 + X_{2j}^2}$.
The contribution of $X_j$ to $\tau \hat{P}_n (Q(x))$ is $1/n$ times the fraction of
the circle $\{ y \in \Re^2 : |y| = R_j\}$ that is in the quadrant $Q(x)$.
There are eight cases to consider:

1. $x_1^2 + x_2^2 > R_j^2$, $x_1, \; x_2 < 0$ or $x_1 < -R_j$ or $x_2 < -R_j$.  
The contribution is 0: the quadrant does not intersect the circle $|y| = R_j$.
1. $x_1, x_2 > R_j$.  
The contribution is $1/n$: the quadrant contains the entire circle $|y| = R_j$.
1. $x_1^2 + x_2^2 \le R_j^2$.  
The quadrant includes an arc that is at most half the circle.  
Let the points at which the quadrant boundary intersects
the circle be $(x_1', x_2)$ and $(x_1, x_2')$.  Then $x_1'$ is the negative
root of $x_1^{'2} = R_j^2 - x_2^2$ and $x_2'$ is the negative root
of $x_2^{'2} = R_j^2 - x_1^2$.
The fraction of the circle included in $Q(x)$ is
$$
\frac{1}{\pi} \sin^{-1} \frac{1}{\sqrt{2}} \left ( 1 + \frac{x_1}{R_j} \sqrt{1 - \frac{x_2^2}{R_j^2}}
+ \frac{x_2}{R_j} \sqrt{1 - \frac{x_1^2}{R_j^2}} \right )^{1/2}.
$$
1. $x_1^2 + x_2^2 > R_j^2$, $-R_j < x_1 \le 0$, $x_2 \ge 0$.  
The fraction of the circle within $Q(x)$ is
$$
q(x_1) \equiv \frac{1}{\pi} \sin^{-1} \frac{1}{\sqrt{2}} \left ( 1 - \frac{x_1^2}{R_j^2} \right )^{1/2}.
$$
1. $x_1^2 + x_2^2 > R_j^2$, $0 \le x_1 < R_j$, $x_2 \ge R_j$.  
The fraction of the circle within $Q(x)$ is $1 - q(x_1)$.
1. $x_1^2 + x_2^2 > R_j^2$, $x_1 \ge 0$, $-R_j < x_2 < 0$.  
The fraction of the circle within $Q(x)$ is $q(x_2)$.
1. $x_1^2 + x_2^2 > R_j^2$, $x_1 \ge R_j$, $0 \le x_2 < R_j$.  
The fraction of the circle within $Q(x)$ is $1-q(x_2)$.
1. $x_1^2 + x_2^2 > R_j^2$, $0 \le x_1 < R_j$, $0 \le x_2 < R_j$.  
The fraction of the circle within $Q(x)$ is $1 - q(x_1) - q(x_2)$.

At which points $x$ should we evaluate the discrepancy 
$ D(x) = |\hat{P}_n(Q(x)) - \tau \hat{P}_n (Q(x))|$?

Let $R = \max_j R_j$.  Then for $x_1, x_2 > R$, $D(x) = 0$.
Similarly, for $x_1, x_2 < -R$, $D(x) = 0$.

We might take $x$ on a fine grid in the square $[-R, R] \times [-R, R]$, but this is wasteful.

Some thought shows that the maximum discrepancy occurs when some datum is just included
in $Q(x)$, which makes $\hat{P}_n$ relatively large compared with $\tau \hat{P}_n$, or when
some datum is just excluded from $Q(x)$,
which makes $\tau \hat{P}_n$ relatively large compared with $\hat{P}_n$.

The possible points of maximum discrepancy are $x$ of the form $(X_{1j}-s\epsilon, X_{2k}-s\epsilon)$ with
$1 \le j, k \le n$, $s \in \{0, 1\}$, and $\epsilon$ small, together with the points
$(X_{1j}-s\epsilon, R)$ and $(R, X_{2j}-s\epsilon)$.

This is a large ($2n^2 + 4n$) but finite number of points.
Denote this set by $\mathcal{X} (\{X_j\}, \epsilon)$.

To draw an iid sample of size $n$ from $\tau \hat{P}_n$, we draw $n$ values
iid uniform on $\{r_j\}_{j=1}^n$ and draw $n$ iid $U[0, 2\pi]$ random variables,
and treat these as the polar coordinates $(r, \theta)$ of $n$ points in $\Re^2$.

To test the null hypothesis of rotational invariance, we would compute
$$
    T(X) = \max_{x \in \Re^k : x \in \mathcal{X} (\{X_j\}, \epsilon)}
    | \hat{P}_n (Q(x)) - \tau \hat{P}_n(Q(x)) |
$$
from the data $X$, then repeatedly draw iid samples $\{X_j^*\}$ of size $n$ from $\tau \hat{P}_n$,
computing
$$
    T(X^*) = \max_{x \in \Re^k : x \in \mathcal{X} (\{X_j^*\}, \epsilon) }
    | \hat{P}_n^* (Q(x)) - \tau \hat{P}_n^*(Q(x)) |
$$
for each.

We would reject the null hypothesis
that $P$ is rotationally invariant (at approximate significance level $\alpha$) if
$T(X)$ exceeds the $1-\alpha$ quantile of the simulated distribution of $T(X^*)$.

Under the null hypothesis, the distribution of the data is invariant under the action
of the rotation group.  

This is not a finite group, so we cannot exhaust the
set of transformations on a computer.  

However, we might consider the subgroup of
rotations by multiples of $2\pi/M$ for some large integer $M$.
We could get an alternative approximate level $\alpha$ test of the hypothesis
of rotational invariance by comparing $T(X)$ with the $1-\alpha$ quantile of
$T$ over all such rotations of the data&mdash;the orbit of the data under this finite
subgroup.

### Application: testing whether seismicity follows a spatially heterogeneous, temporally homogeneous Poisson Process

Reference: Luen and Stark, 2012. http://onlinelibrary.wiley.com/doi/10.1111/j.1365-246X.2012.05400.x/pdf

Poisson process: condition on number of events. Given the number of events, temporal distribution uniform, times exchangeable.

What's the "projection" operation?



In [1]:
%run talkTools.py